In [3]:
#importing dependencies
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np


#reading dataset and sorted title list
imdb = pd.read_csv('dataset/netflix.csv')
titles_list = imdb['sortedTitle']



#vectorization
cv = CountVectorizer(dtype=np.uint8)
dtm = cv.fit_transform(imdb['genres']).toarray()
new_matrix = np.concatenate((dtm, np.array(imdb['averageRating']).reshape(-1, 1)), axis=1)



#minmax scaling
MMS = MinMaxScaler()
numVotes = np.array(imdb['numVotes'])
numVotes = numVotes.reshape(-1, 1)
numVotes = MMS.fit_transform(numVotes)
new_matrix = np.concatenate((new_matrix, numVotes), axis=1)


#cosine similarity to find similarity between features 
similarities = cosine_similarity(new_matrix, dense_output=False)


def build_recommendations(title):
    try:
        title = title.lower()
        averageRating = averageRating.lower()
        tv_shows = ['tvSeries', 'tvMovie', 'tvMiniSeries', 'video', 'tvSpecial']
        sorted_title_found = True in [True for t in imdb['sortedTitle'] if t.lower() == title]
        if sorted_title_found:
            idx = imdb[imdb['sortedTitle'].apply(lambda x: x.lower()) == title].index[0]
        else:
            idx = imdb[imdb['primaryTitle'].apply(lambda x: x.lower()) == title].index[0]

        recommendations = imdb['sortedTitle'].iloc[similarities[idx].argsort()[::-1]][0:500]

        if imdb.iloc[idx]['titleType'] in tv_shows:
            tv_recommendations = {rec: [imdb['tconst'].iloc[rec], imdb['sortedTitle'].iloc[rec]] for rec in
                                  recommendations.index if imdb['titleType'].iloc[rec] in tv_shows}
            return pd.DataFrame(tv_recommendations).transpose().iloc[1:11]

        else:
            movie_recommendations = {rec: [imdb['tconst'].iloc[rec], imdb['sortedTitle'].iloc[rec]] for rec in
                                     recommendations.index if imdb['titleType'].iloc[rec] == 'movie'}
            return pd.DataFrame(movie_recommendations).transpose().iloc[1:11]
    except:
        return None


def get_recommendations(title):
    recommendations = build_recommendations(title)
    if recommendations is None:
        return recommendations
    else:
        recommendations.rename(columns={0: 'tconst', 1: 'title'}, inplace=True)
        recommendations.reset_index(drop=True, inplace=True)
        recommendations['urls'] = [f'https://www.imdb.com/title/{title_id}/'for title_id in recommendations['tconst']]
        return recommendations.drop('tconst', axis=1)


def get_movie_data():
    return titles_list


In [4]:
import tkinter.messagebox
import webbrowser
import pygame
import time
from functools import partial
from tkinter import ttk
import imdb_recommendation_system as ims
from tkHyperlinkManager import *


def play_menu_sound(option):

    if option == 'menu_bar':
        pygame.mixer.music.load('music/button-11.wav')
        pygame.mixer.music.play()
    elif option == 'quit':
        pygame.mixer.music.load('music/quit.wav')
        pygame.mixer.music.play()
        time.sleep(0.3)
        root.destroy()



def update_values():
    filter_str = combo1.get().lower()
    filter_str = ' '.join([word for word in re.split(r'\s+', filter_str) if word != ''])  # handling white space
    # if no input is provided show the entire database
    if filter_str == '':
        combo1['values'] = movie_data
    # else filter based on the input
    else:
        filtered_list_1 = []  # holds values that starts with the input string
        filtered_list_2 = []  # holds values that matches the input pattern in the database
        for value in movie_data:
            if value.lower().startswith(filter_str):
                filtered_list_1.append(value)
            elif filter_str in value.lower():
                filtered_list_2.append(value)
        combo1['values'] = filtered_list_1 + filtered_list_2 


def open_link(my_url):
    webbrowser.open_new(url=my_url)


    
def get_text(event=None):
    
    text_widget = Text(frame, font='Helvetica 13 italic', cursor='arrow', bg='#FFFAFA', height=11, width=73)
    hyperlink = HyperlinkManager(text_widget)
    
    text_widget.tag_configure('tag-center', justify='center')
    text_widget.tag_configure('tag-left', justify='left')
    #get input from combo widget
    query = combo1.get() 
    query = ' '.join([word for word in re.split(r'\s+', query) if word != ''])
    text = ims.get_recommendations(query)
    
    
    #if the movie/tv show not found print some tips
    if text is None:  
        text = "\n\n\nItem not found!\n"
        text_widget.insert(1.0, text, 'tag-center')
        text_widget.insert(END, '\nTry Searching again', 'tag-center')
        
    else: 
        text_widget.delete(1.0, END)  
        for idx, title,imdb_url in text.itertuples():  
            text_widget.insert(END, title, hyperlink.add(partial(open_link, imdb_url)))  
            if idx != 9:  
                text_widget.insert(END, '\n')
                text_widget.insert(END, '\n')
    text_widget.config(highlightcolor='black', highlightbackground="black", highlightthickness=2)
    text_widget.place(x=370, y=360)
    
    # adding scrollbar to the text widget
    scroll_y = Scrollbar(text_widget, orient='vertical', command=text_widget.yview)
    scroll_y.place(x=185*3 + 30, relheight=1)
    text_widget.configure(state='disabled', yscrollcommand=scroll_y.set) 

    

#initialize master window
root = Tk()
root.configure(bg='black')
root.title("Netflix Recommendation System")
root.geometry('1450x768')  
root.resizable(width=True, height=True) 


#Netflix logo image
frame = Frame(root).place(x=50, y=25)
frame_bg_image = PhotoImage(file=r'logo.png')
frame_label = Label(frame,height="150",width="250", image=frame_bg_image)
frame_label.pack()

frame1 = Frame(root).place(x=150, y=25)
frame1_bg_image = PhotoImage(file=r'netflix.png')
frame1_label = Label(frame, height="500",width="1200", image=frame1_bg_image)
frame1_label.pack()
# label2 = Label(frame, font='Helvetica 20 italic', text='Recommendation System', height=2, width=20,
#                bg='black', highlightthickness=2, highlightbackground="black",foreground="red")

# label2.place(x=500, y=120)










#creating widgets
label1 = Label(frame, font='Helvetica 13 italic', text='Select a Movie/Series!', height=1, width=65,
               bg='#c21a09', highlightthickness=2)

movie_data = ims.get_movie_data()  # get the database of all the movies/tv shows
combo1 = ttk.Combobox(frame, width=64, font=("Courier", 13), postcommand=update_values, values=movie_data)

button1 = Button(frame, text='GO!', font='Helvetica 13 bold italic', bg='#800000', width=42, command=get_text)

def instructions_text():
    instructions_text = Text(frame, font='Helvetica 13 italic', cursor='arrow', bg='#FFFAFA', height=11, width=73)



# print instructions in the text widget
    instructions_text.tag_configure('tag-center', justify='center')
    instructions_text.tag_configure('tag-center-underline', justify='center', underline=1)
    instructions_text.tag_configure('tag-left', justify='left')
    instructions_text.insert(1.0, '\nWelcome to Netflix recommendation!\n', 'tag-center')
    instructions_text.insert(END, "\nInstructions\n", 'tag-center-underline')
    instructions_text.insert(END, "\n 1. Enter the keywords of a Movie/Series you want to search. \n 2. Or Select from the "
                                  "dropdown menu. \n 3. Press ENTER or 'GO!' to search. \n ", 'tag-center')

    instructions_text.config(highlightcolor='black', highlightbackground="black", highlightthickness=2)
    instructions_text.place(x=370, y=360)
    instructions_text.configure(state='disabled')

# placing widgets
button2 = Button(frame, text='Instructions', font='Helvetica 13 bold italic', bg='#800000', width=15, command=instructions_text)  
label1.place(x=370, y=200)
root.option_add('*TCombobox*Listbox.font', ("Courier", 13))
combo1.place(x=370, y=263, height=32)
button1.place(x=480, y=310)
button2.place(x=1150, y=30)
combo1.bind('<Return>', get_text)


# main loop
if __name__ == '__main__':
    pygame.mixer.init()
    root.mainloop()
